# Example State Machine Usage



In [1]:
from state_machine import StateMachine, State, Model, Condition, AlwaysTrue, AlwaysFalse

In [2]:
class ReceptionistModel( Model ):
    def __init__( self, name='Bernie' ):
        super( ReceptionistModel, self ).__init__( name )
        
model = ReceptionistModel()

Make some conditions. Not that PhoneRings listens for 'phone', and CustomerWants listerns for 'customer'.

In [3]:
class PhoneRings(Condition):
    listens_for='phone'
    def __init__(self):
        self.name='Phone Rings'
    def is_triggered(self, data, model):
        return data=='ring ring'
PHONE_RINGS = PhoneRings()
assert PHONE_RINGS.is_triggered( 'ring ring', None )==True
assert PHONE_RINGS.is_triggered( 'silence', None )==False

In [4]:
class CustomerWants(Condition):
    listens_for='customer'
    def __init__(self, wants):
        self.name='Wants %s'%wants
        self.wants=wants
        
    def is_triggered(self, data, model):
        if isinstance(data, dict):
            return self.wants==data.get('wants', self.wants)

CUSTOMER_WANTS_PIZZA   = CustomerWants('pizza')
CUSTOMER_WANTS_LASAGNE = CustomerWants('lasagne')

assert CUSTOMER_WANTS_PIZZA.is_triggered( {'wants':'lasagne'}, model )==False
assert CUSTOMER_WANTS_PIZZA.is_triggered( {'wants':'pizza'}, model )==True

In [5]:
class EnterPizzaOrder(State):
    def on_start(self, data, model):
        model.logger.info('Entering Pizza Order')
        
class EnterLasagneOrder(State):
    def on_start(self, data, model):
        model.logger.info('Entering Lasagne Order')

In [6]:
receptionist = StateMachine( 'Receptionist' )

wait_for_phone    = State('Wait For Phone')
take_order        = State('Take Order')

enter_lasagne_order = EnterLasagneOrder( 'Enter Lasagne Order' )
enter_pizza_order   = EnterPizzaOrder('Enter Pizza Order' )

wait_for_phone.add_transition_to(take_order,PHONE_RINGS)
take_order.add_transition_to( enter_lasagne_order, CUSTOMER_WANTS_LASAGNE )
take_order.add_transition_to( enter_pizza_order, CUSTOMER_WANTS_PIZZA )

### Receptionist waits, but only receives silence

Broadcast silence on the phone.

In [7]:
data = { 'phone':'SILENCE' }
model.set_state( wait_for_phone )
receptionist.run( data, model )
assert wait_for_phone==model.current_state

INFO:default:phone, SILENCE
INFO:default:We have 1 transitions
INFO:default:TEsting Transition from State:Wait For Phone to State:Take Order, Condition:(Phone Rings)


### Phone rings, so receptionist takes order

Broadcast a ringing phone.

In [8]:
data = { 'phone':'ring ring' }
model.set_state( wait_for_phone )
receptionist.run(data, model)
assert take_order==model.current_state

INFO:default:phone, ring ring
INFO:default:We have 1 transitions
INFO:default:TEsting Transition from State:Wait For Phone to State:Take Order, Condition:(Phone Rings)
INFO:default:Returning Transition from State:Wait For Phone to State:Take Order, Condition:(Phone Rings)
INFO:default:phone, ring ring
INFO:default:We have 0 transitions


### Customer Wants Pizza

Take an order from a customer who wants pizza.

In [9]:
data = {'customer':{'wants':'pizza'}}
model.set_state( take_order )
receptionist.run(data,model)
assert enter_pizza_order==model.current_state

INFO:default:customer, {'wants': 'pizza'}
INFO:default:We have 2 transitions
INFO:default:TEsting Transition from State:Take Order to State:Enter Lasagne Order, Condition:(Wants lasagne)
INFO:default:TEsting Transition from State:Take Order to State:Enter Pizza Order, Condition:(Wants pizza)
INFO:default:Returning Transition from State:Take Order to State:Enter Pizza Order, Condition:(Wants pizza)
INFO:default:Entering Pizza Order
INFO:default:customer, {'wants': 'pizza'}
INFO:default:We have 0 transitions


### Customer Wants Lasagne

Take an order from a customer who wants pizza.

In [10]:
data = {'customer':{'wants':'lasagne'}}
model.set_state( take_order )
receptionist.run(data,model)
assert enter_lasagne_order==model.current_state

INFO:default:customer, {'wants': 'lasagne'}
INFO:default:We have 2 transitions
INFO:default:TEsting Transition from State:Take Order to State:Enter Lasagne Order, Condition:(Wants lasagne)
INFO:default:Returning Transition from State:Take Order to State:Enter Lasagne Order, Condition:(Wants lasagne)
INFO:default:Entering Lasagne Order
INFO:default:customer, {'wants': 'lasagne'}
INFO:default:We have 0 transitions


### The Full Pizza Sequence

Test a ringing phone, followed by a customer who wants pizza.

In [11]:
model.set_state( wait_for_phone )
receptionist.run( {'phone':'SILENCE'}, model )
receptionist.run( {'phone':'ring ring'}, model)
receptionist.run({'customer':{'wants':'pizza'}}, model)
assert enter_pizza_order==model.current_state

INFO:default:phone, SILENCE
INFO:default:We have 1 transitions
INFO:default:TEsting Transition from State:Wait For Phone to State:Take Order, Condition:(Phone Rings)
INFO:default:phone, ring ring
INFO:default:We have 1 transitions
INFO:default:TEsting Transition from State:Wait For Phone to State:Take Order, Condition:(Phone Rings)
INFO:default:Returning Transition from State:Wait For Phone to State:Take Order, Condition:(Phone Rings)
INFO:default:phone, ring ring
INFO:default:We have 0 transitions
INFO:default:customer, {'wants': 'pizza'}
INFO:default:We have 2 transitions
INFO:default:TEsting Transition from State:Take Order to State:Enter Lasagne Order, Condition:(Wants lasagne)
INFO:default:TEsting Transition from State:Take Order to State:Enter Pizza Order, Condition:(Wants pizza)
INFO:default:Returning Transition from State:Take Order to State:Enter Pizza Order, Condition:(Wants pizza)
INFO:default:Entering Pizza Order
INFO:default:customer, {'wants': 'pizza'}
INFO:default:We hav

### The Full Pizza Sequence

Test a ringing phone, followed by a customer who wants lasagne.

In [12]:
model.set_state( wait_for_phone )
receptionist.run( {'phone':'SILENCE'}, model )
receptionist.run( {'phone':'ring ring'}, model)
receptionist.run({'customer':{'wants':'lasagne'}}, model)
assert enter_lasagne_order==model.current_state

INFO:default:phone, SILENCE
INFO:default:We have 1 transitions
INFO:default:TEsting Transition from State:Wait For Phone to State:Take Order, Condition:(Phone Rings)
INFO:default:phone, ring ring
INFO:default:We have 1 transitions
INFO:default:TEsting Transition from State:Wait For Phone to State:Take Order, Condition:(Phone Rings)
INFO:default:Returning Transition from State:Wait For Phone to State:Take Order, Condition:(Phone Rings)
INFO:default:phone, ring ring
INFO:default:We have 0 transitions
INFO:default:customer, {'wants': 'lasagne'}
INFO:default:We have 2 transitions
INFO:default:TEsting Transition from State:Take Order to State:Enter Lasagne Order, Condition:(Wants lasagne)
INFO:default:Returning Transition from State:Take Order to State:Enter Lasagne Order, Condition:(Wants lasagne)
INFO:default:Entering Lasagne Order
INFO:default:customer, {'wants': 'lasagne'}
INFO:default:We have 0 transitions
